In [11]:
#### import pandas as pd
import tkinter as tk
from tkinter import ttk
# from PIL import Image, ImageOps
# from pillow_heif import register_heif_opener
from docx import Document
from docx.shared import Inches
from docx.shared import Pt
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from lxml import etree
from docx.enum.table import WD_TABLE_ALIGNMENT
import traceback
from docx.enum.text import WD_ALIGN_PARAGRAPH
import pandas as pd



def format_client(input_text):
    # Split the text at every occurrence of "and" (case-insensitive)
    parts = input_text.split('and')
    
    # Combine parts with "and" followed by a newline after each part
    formatted_text = (' and\n'.join(part.strip() for part in parts)).strip()
    
    return formatted_text

def format_address(address):
    # Split the address into components based on known positions
    parts = address.split()
    
    # Assuming the address format is always correct and has a space between components
    street_address = ' '.join(parts[:-3])  # All but last three elements
    city = parts[-3]
    state = parts[-2]
    zip_code = parts[-1]
    
    # Format the address
    formatted_address = f"{street_address}\n{city} {state} {zip_code}"
    
    return formatted_address

def city_state_zip(address):

    parts = address.split()
    city = parts[-3]
    state = parts[-2]
    zip_code = parts[-1]
    
    # Get city, state, zip
    get_city_state_zip = f"{city} {state} {zip_code}"

    return get_city_state_zip


def extract_street_address(address):
    # Split the address into parts based on spaces
    parts = address.split()
    
    # Ensure there are at least three parts to avoid errors
    if len(parts) < 3:
        raise ValueError("Address must contain at least a street address, city, state, and zip code.")
    
    # Extract the last three parts: city, state, and zip code
    city = parts[-3]
    state = parts[-2]
    zip_code = parts[-1]
    
    # Combine the remaining parts to form the street address
    street_address = ' '.join(parts[:-3])
    
    # Return the street address
    return street_address

# Define containers for user input
client = input("Enter the client's name: ")
client_mailing_address = input("Enter the client's mailing address: ")
property_address = input("Enter the property address: ")
subject_county = input("Enter the county of the subject property: ")
report_date = input("Enter the report date: ")
prg_ref_num = input("Enter the PRG reference number: ")
tax_account = input("Enter the tax account: ")

def lot_size_function():
    print("Please choose a unit of measurement for lot size:")
    print("1. Acres")
    print("2. Square Feet")
    choice = input("Enter the number of your choice (1 or 2): ").strip()
    if choice == "1":
        lot_size = input("Enter the number of acres: ").strip()
        lot_unit = "acres"
    elif choice == "2":
        lot_size = input("Enter the number of lot square feet: ").strip()
        lot_unit = "square feet"
    else:
        print("Invalid choice. Please enter 1 for Acres or 2 for Square Feet.")
        return None, None  # Return None for both if the choice is invalid
    
    return lot_size, lot_unit  # Return the values

# lot_size_function()
lot_size, lot_unit = lot_size_function()

def parcel_shape_function():
    print("What is the shape of the parcel?")
    print("1. Rectangular")
    print("2. Irregular")
    choice = input("Please enter the number corresponding to your choice: ").strip()
    if choice == "1":
        parcel_shape = "rectangular"
    elif choice == "2":
        parcel_shape = "irregular"
    else:
        print("Invalid choice. Please enter 1 for Rectangular or 2 for Irregular.")
    return parcel_shape

parcel_shape = parcel_shape_function()

def vacant_or_improved():
    print("Is the subject property vacant or improved? ")
    print("1. Vacant")
    print("2. Improved")
    choice = input("Enter the number of your choice: ").strip()
    if choice == "1":
        vac_or_imp = "vacant"
        print(f"The property is {vac_or_imp}.")
    elif choice == "2":
        vac_or_imp = "improved"
        building_sf = input("Please enter the building square feet: ").strip()
        year_built = input("Enter the year built: ").strip()
        print("Please choose a building type:")
        print("1. Residence")
        print("2. Flex")
        print("3. Mixed-Use")
        building_type_choice = input("Enter the number corresponding to your choice: ").strip()
        if building_type_choice == "1":
            stories = input("Enter the number of stories: ").strip()
            bedrooms = input("Enter the number of bedrooms: ").strip()
            full_bath = input("Enter the number of full bathrooms: ").strip()
            half_bath = input("Enter the number of half bathrooms: ").strip()
            property_type = "residence"
        elif building_type_choice == "2":
            property_type = "flex"
        elif building_type_choice == "3":
            property_type = "mixed-use"
        else:
            print("Invalid choice. Please enter the number corresponding to your choice.")
    return vac_or_imp, building_sf, year_built, stories, bedrooms, full_bath, half_bath, property_type # Return the values

vac_or_imp, building_sf, year_built, stories, bedrooms, full_bath, half_bath, property_type = vacant_or_improved()
            
liber = input("Enter the deed Liber: ")
folio = input("Enter the deed Folio: ")
map_num = input("Enter the map number: ")
grid = input("Enter the grid number: ")
parcel = input("Enter the parcel number: ")
election_district = input("Enter the election district: ")
census_tract = input("Enter the census tract: ")
road_frontage = input("Enter the total road frontage of the subject property (feet): ")

def choose_value_type():
    global valuation_choice, decedent, date_of_valuation

    # Display the options to the user
    print("Please choose type of valuation:")
    print("1. Current Market Value 'As-Is'")
    print("2. Retrospective Market Value")

    # Loop until a valid choice is made
    while True:
        choice = input("Enter the number of your choice (1 or 2): ").strip()
        if choice == "1":
            # Set the global variables
            valuation_choice = "Current Market Value 'As-Is'"
            decedent = None
            date_of_valuation = input("Enter the date of valuation: ")
            break
        elif choice == "2":
            # Prompt for additional details
            valuation_choice = "Retrospective Market Value"
            decedent = input("Please enter the name of the decedent: ").strip()
            date_of_valuation = input("Please enter the date of death (e.g., YYYY-MM-DD): ").strip()
            break
        else:
            print("Invalid choice. Please enter 1 for Current Market Value 'As-Is' or 2 for Retrospective Market Value.")

# Call the function to populate the containers
choose_value_type()

file_path = "Zoning_Workbook.xlsx"
    
# Read the Excel file into a DataFrame
df = pd.read_excel(file_path)

# Show the values in the first column to the user
first_column_values = df.iloc[:, 0].unique()
print("Please the subject's zoning value from the following list:")
for idx, value in enumerate(first_column_values, start=1):
    print(f"{idx}. {value}")

# Prompt the user to select a value
selection = int(input("\nEnter the number corresponding to your choice: "))
selected_value = first_column_values[selection]

# Create a new DataFrame with the row(s) corresponding to the user's selection
new_df = df[df.iloc[:, 0] == selected_value]\

zone = new_df.iloc[0, 0]
zone_abr = new_df.iloc[0, 1]
ib = new_df.iloc[0, 2]
zone_desc = new_df.iloc[0, 3]

# Load the base template
doc = Document('Base Template.docx')

# Set the desired margins (in inches)
left_margin = Inches(1)
right_margin = Inches(1)

# Apply margins to all sections in the document
for section in doc.sections:
    section.left_margin = left_margin
    section.right_margin = right_margin

# Add space before title paragraph
doc.add_paragraph()  # Adds a blank paragraph for spacing

##### Title Page

# Add a title paragraph with centered text
title_paragraph = doc.add_paragraph()
title_run = title_paragraph.add_run("APPRAISAL REPORT")

# Set font properties
title_run.bold = True
title_run.font.size = Pt(36)
title_run.font.name = 'Times New Roman'

# Set paragraph alignment to center
title_paragraph.alignment = 1  # Center alignment

doc.add_paragraph()  # Add a blank paragraph for spacing before the image

# Define the left column values and corresponding user input variables
left_column_values = ["CLIENT", "PROPERTY ADDRESS", "REPORT DATE", "PRG REFERENCE NUMBER"]
right_column_values = [format_client(client), format_address(property_address), report_date, prg_ref_num]

# Function to add a table with one row and two columns
def add_table(doc, left_text, right_text):
    table = doc.add_table(rows=1, cols=2)
    table.autofit = True
    table.style = 'Table Grid'
    
    # Access the first row
    row = table.rows[0]

    # Set column widths
    for cell in row.cells:
        cell.width = Inches(2.5)  # Adjust the width as needed

    # Center alignment of each table
    table.alignment = WD_TABLE_ALIGNMENT.CENTER
    
    # Set the text for the left column
    cell_left = row.cells[0]
    cell_left.text = left_text
    cell_left.paragraphs[0].runs[0].font.size = Pt(12)
    cell_left.paragraphs[0].runs[0].font.name = 'Times New Roman'
    cell_left.paragraphs[0].runs[0].font.bold = True
    cell_left.paragraphs[0].runs[0].text = cell_left.paragraphs[0].runs[0].text.upper()

    # Set the text for the right column
    cell_right = row.cells[1]
    cell_right.text = right_text
    cell_right.paragraphs[0].runs[0].font.size = Pt(12)
    cell_right.paragraphs[0].runs[0].font.name = 'Times New Roman'
    cell_right.paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.LEFT
    
    # Remove borders from the table
    tbl = table._tbl
    tblPr = tbl.tblPr
    tblBorders = OxmlElement('w:tblBorders')
    for border in ['top', 'left', 'bottom', 'right', 'insideH', 'insideV']:
        border_elem = OxmlElement(f'w:{border}')
        border_elem.set(qn('w:val'), 'nil')
        tblBorders.append(border_elem)
    tblPr.append(tblBorders)
    
    return table

# Populate tables and add them to the document
for i in range(len(left_column_values)):
    add_table(doc, left_column_values[i], right_column_values[i])
    
    # Add a blank paragraph for spacing between tables
    doc.add_paragraph()
    
# table.alignment = WD_TABLE_ALIGNMENT.CENTER

# Add the image after the table
doc.add_paragraph()  # Add a blank paragraph for spacing before the image

# Add the image after the table
doc.add_paragraph()  # Add a blank paragraph for spacing before the image

# # Register HEIC opener with Pillow
# register_heif_opener()

# # Function to convert HEIC to JPG
# def convert_heic_to_jpg(heic_file_path, jpg_file_path):
#     # Open HEIC file using Pillow
#     image = Image.open(heic_file_path)
    
#     # Save as JPG
#     image.save(jpg_file_path, format='JPEG')

# # File paths
# heic_file_path = 'IMG_1303.HEIC'
# jpg_file_path = 'IMG_1303.jpg'

# # Convert HEIC to JPG
# convert_heic_to_jpg(heic_file_path, jpg_file_path)

# # Create a paragraph for centering the image
# image_paragraph = doc.add_paragraph()
# image_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER  # Center alignment

# # Add the image to the document
# # Resize the image (example size: 5 inches wide by 4 inches high)
# image_run = image_paragraph.add_run().add_picture(jpg_file_path, width=Inches(5), height=Inches(4))

# # Add the image after the table
# doc.add_paragraph()  # Add a blank paragraph for spacing before the image

# Create a new paragraph for the address
address_paragraph = doc.add_paragraph()
address_paragraph.alignment = 1  # Center alignment

# Add address text to the paragraph
address_run = address_paragraph.add_run(property_address)

# Set font properties for the address
address_run.bold = True
address_run.italic = True
address_run.font.size = Pt(12)
address_run.font.name = 'Times New Roman'

# Add the image after the table
doc.add_paragraph()  # Add a blank paragraph for spacing before the image
doc.add_paragraph()

# Add the PRG Logo image to the same paragraph
logo_paragraph = doc.add_paragraph()
logo_paragraph.alignment = 1  # Center alignment

# Add the PRG Logo image to the paragraph
logo_paragraph.add_run().add_picture('PRG Logo.jpg', width=Inches(2.0))  # Adjust width as needed

# Add a page break
doc.add_page_break()



#### Page 2



doc.add_paragraph()  # Adds a blank paragraph for spacing
doc.add_paragraph()  # Adds a blank paragraph for spacing
doc.add_paragraph()  # Adds a blank paragraph for spacing
doc.add_paragraph()  # Adds a blank paragraph for spacing

# Add a title paragraph with centered text
title_paragraph = doc.add_paragraph()
title_run = title_paragraph.add_run("APPRAISAL REPORT")
title_run.bold = True
title_run.font.size = Pt(14)
title_run.font.name = 'Times New Roman'
title_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

doc.add_paragraph()  # Adds a blank paragraph for spacing
doc.add_paragraph()  # Adds a blank paragraph for spacing

# Add a title paragraph with centered text
title_paragraph = doc.add_paragraph()
title_run = title_paragraph.add_run("ADDRESS")
title_run.bold = True
title_run.font.size = Pt(12)
title_run.font.name = 'Times New Roman'
title_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

doc.add_paragraph()  # Adds a blank paragraph for spacing

# Creates "Address" section
paragraph = doc.add_paragraph(format_address(property_address))
paragraph.alignment = 1

doc.add_paragraph()  # Adds a blank paragraph for spacing
doc.add_paragraph()  # Adds a blank paragraph for spacing

# Add a title paragraph with centered text
legal_desc_paragraph = doc.add_paragraph()
legal_desc_run = legal_desc_paragraph.add_run("LEGAL DESCRIPTION")
legal_desc_run.bold = True
legal_desc_run.font.size = Pt(12)
legal_desc_run.font.name = 'Times New Roman'
legal_desc_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

doc.add_paragraph()  # Adds a blank paragraph for spacing

# Creates legal description
par1 = doc.add_paragraph(f"Tax Account: {tax_account}")
par1.alignment = 1
par2 = doc.add_paragraph(f"{lot_size} {lot_unit} of land")
par2.alignment = 1
par3 = doc.add_paragraph(f"Liber {liber}/Folio {folio}")
par3.alignment = 1
par4 = doc.add_paragraph(f"Map {map_num}/Grid {grid}/Parcel {parcel}")
par4.alignment = 1
par5 = doc.add_paragraph(f"{election_district} Election District")
par5.alignment = 1
par6 = doc.add_paragraph(f"Census Tract: {census_tract}")
par6.alignment = 1
par7 = doc.add_paragraph(f"{city_state_zip(property_address)}")
par7.alignment = 1

doc.add_paragraph()  # Adds a blank paragraph for spacing
doc.add_paragraph()  # Adds a blank paragraph for spacing

# Add a title paragraph with centered text
as_of_paragraph = doc.add_paragraph()
as_of_run = as_of_paragraph.add_run("AS OF")
as_of_run.bold = True
as_of_run.font.size = Pt(12)
as_of_run.font.name = 'Times New Roman'
as_of_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

doc.add_paragraph()  # Adds a blank paragraph for spacing

# Creates the "As Of" section
par8 = doc.add_paragraph(f"{date_of_valuation}")
par8.alignment = 1
par9 = doc.add_paragraph(f"{valuation_choice}")
par9.alignment = 1

doc.add_paragraph()  # Adds a blank paragraph for spacing
doc.add_paragraph()  # Adds a blank paragraph for spacing

# Add a title paragraph with centered text
as_of_paragraph = doc.add_paragraph()
as_of_run = as_of_paragraph.add_run("FOR")
as_of_run.bold = True
as_of_run.font.size = Pt(12)
as_of_run.font.name = 'Times New Roman'
as_of_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

doc.add_paragraph()  # Adds a blank paragraph for spacing

# Creates "For" section
par10 = doc.add_paragraph(f"{client}")
par10.alignment = 1
par11 = doc.add_paragraph(f"{format_address(client_mailing_address)}")
par11.alignment = 1

doc.add_paragraph()  # Adds a blank paragraph for spacing
doc.add_paragraph()  # Adds a blank paragraph for spacing

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("PREPARED BY")
prep_run.bold = True
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

doc.add_paragraph()  # Adds a blank paragraph for spacing

# Creates the "Prepared By" section
par12 = doc.add_paragraph(f"{prepared_by}")
par12.alignment = 1

doc.add_paragraph()  # Adds a blank paragraph for spacing
doc.add_paragraph()  # Adds a blank paragraph for spacing

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("Copyright © 2024")
prep_run.bold = True
prep_run.font.size = Pt(10)
prep_run.font.name = 'Calibri'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("Pugh Real Estate Group, LLC")
prep_run.bold = True
prep_run.font.size = Pt(12)
prep_run.font.name = 'Calibri'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("Real Estate Appraisers")
prep_run.bold = True
prep_run.font.size = Pt(10)
prep_run.font.name = 'Calibri'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("P.O. Box 378")
prep_run.bold = True
prep_run.font.size = Pt(10)
prep_run.font.name = 'Calibri'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("Frederick, Maryland 21705")
prep_run.bold = True
prep_run.font.size = Pt(10)
prep_run.font.name = 'Calibri'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Add a page break
doc.add_page_break()



#### Page 3



# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("PUGH REAL ESTATE GROUP, LLC")
prep_run.bold = False
prep_run.font.size = Pt(18)
prep_run.font.name = 'Copperplate Gothic Bold'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("REAL ESTATE APPRAISERS")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Copperplate Gothic Bold'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("POST BOX 378")
prep_run.bold = False
prep_run.font.size = Pt(8)
prep_run.font.name = 'Copperplate Gothic Bold'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("FREDERICK, MARYLAND 21705")
prep_run.bold = False
prep_run.font.size = Pt(8)
prep_run.font.name = 'Copperplate Gothic Bold'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("(301) 898-1178 ")
prep_run.bold = False
prep_run.font.size = Pt(8)
prep_run.font.name = 'Copperplate Gothic Bold'
prep_run.font.underline = True 
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

doc.add_paragraph()  # Adds a blank paragraph for spacing

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run(f"{report_date}")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.RIGHT

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run(f"{client}")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run(f"{format_address(client_mailing_address)}")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT

doc.add_paragraph()  # Adds a blank paragraph for spacing

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run(f"Re:     {extract_street_address(property_address)}")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run(f"           {city_state_zip(property_address)}")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT

doc.add_paragraph()  # Adds a blank paragraph for spacing

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run(f"Dear {client},")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT

doc.add_paragraph()  # Adds a blank paragraph for spacing

def add_paragraph_based_on_choice():
    global valuation_choice, decedent, date_of_death, report_date

    # Add the title paragraph with centered text based on the valuation choice
    if valuation_choice == "Current Market Value 'As-Is'":
        # Add a title paragraph with centered text
        prep_paragraph = doc.add_paragraph()
        prep_run = prep_paragraph.add_run(
            f"""You asked me to provide you with an appraisal value for the property identified above. This appraisal report provides an opinion of value as of {report_date}, which was the date of inspection. The attached report provides essential data and detailed reasoning employed in reaching my opinion of value."""
        )
        prep_run.bold = False
        prep_run.font.size = Pt(12)
        prep_run.font.name = 'Times New Roman'
        prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT

    elif valuation_choice == "Retrospective Market Value":
        # Add a title paragraph with centered text
        prep_paragraph = doc.add_paragraph()
        prep_run = prep_paragraph.add_run(
            f"""Recently, you asked me to provide you with a retrospective market value for the property identified above. This appraisal report provides an opinion of value as of {date_of_death}, which is the date of death of {decedent}. The attached appraisal report provides essential data and detailed reasoning employed in reaching my opinion of value."""
        )
        prep_run.bold = False
        prep_run.font.size = Pt(12)
        prep_run.font.name = 'Times New Roman'
        prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT

add_paragraph_based_on_choice()

doc.add_paragraph()  # Adds a blank paragraph for spacing

prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("SUMMARY OF SITE, ZONING, AND IMPROVEMENTS")
prep_run.bold = True
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT

doc.add_paragraph()  # Adds a blank paragraph for spacing

prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run(f"The subject property is located on a {parcel_shape}-shaped parcel ({parcel}) that encompasses {lot_size} {lot_unit}, according to the land records of {subject_county} County. This site features approximately {road_frontage} feet of frontage along FILLER STREET")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT

doc.add_paragraph()  # Adds a blank paragraph for spacing

def add_paragraph_based_on_property_type():
    global stories, bedrooms, full_bath, half_bath, property_type, building_sf, year_built

    if property_type == "residence" and half_bath not in ["zero",0,"0","Zero"]:
        prep_paragraph = doc.add_paragraph()
        prep_run = prep_paragraph.add_run(
            f"""This parcel includes a {stories}-story residence building with {bedrooms} bedrooms, {full_bath} full bathrooms and {half_bath} half bathrooms. The building has {building_sf} of gross living area and was constructed in {year_built} (according to the {subject_county} county land records).""" 
        )
        prep_run.bold = False
        prep_run.font.size = Pt(12)
        prep_run.font.name = 'Times New Roman'
        prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT

    elif valuation_choice == "residence" and half_bath in ["zero",0,"0","Zero"]:
        prep_paragraph = doc.add_paragraph()
        prep_run = prep_paragraph.add_run(
            f"""This parcel includes a {stories}-story residence building with {bedrooms} bedrooms and {full_bath} bathrooms. The building has {building_sf} of gross living area and was constructed in {year_built} (according to the {subject_county} county land records.)"""
        )
        prep_run.bold = False
        prep_run.font.size = Pt(12)
        prep_run.font.name = 'Times New Roman'
        prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT

    else:
        prep_paragraph = doc.add_paragraph()
        prep_run = prep_paragraph.add_run(
            "FILLER PARAGRAPH"
        )
        prep_run.bold = False
        prep_run.font.size = Pt(12)
        prep_run.font.name = 'Times New Roman'
        prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT

add_paragraph_based_on_property_type()

doc.add_paragraph()  # Adds a blank paragraph for spacing

prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run(f"The subject property is zoned {zone} ({zone_abr}) by the {ib}. {zone_desc}")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT

# Add a page break
doc.add_page_break()



#### Page 4



prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("EXTRAORDINARY ASSUMPTIONS")
prep_run.bold = True
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT

prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("Uniform Standard of Professional Appraisal Practice (USPAP) defines an Extraordinary Assumption as, “an assumption, directly related to a specific assignment, as of the effective date of the assignment result, which, if found to be false, could alter the appraiser’s opinions or conclusions.” ")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.space_before = Pt(6)

prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("No extraordinary assumptions were made for this assignment.")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_run.italic = True
prep_paragraph.space_before = Pt(6)

prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("HYPOTHETICAL CONDITIONS")
prep_run.bold = True
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.space_before = Pt(6)

prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("USPAP defines a Hypothetical Condition as, “that which is contrary to what is known the appraiser to exist on the effective date of the assignment results but is used for the purpose of analysis.” ")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.space_before = Pt(6)

prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("No hypothetical conditions were made for this assignment.")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_run.italic = True
prep_paragraph.space_before = Pt(6)

prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("After careful consideration of all the factors which create value, it is my opinion that the subject property has a value as follows:")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.space_before = Pt(6)



prepared_by = input("Prepared by: ")

# Save the document
doc.save("appraisal_report.docx")

Enter the client's name:  Mr. Bob Smith
Enter the client's mailing address:  lkj saldkfj jfdk laskd asjdk
Enter the property address:  lksdf fejk sldk fjek sldk jsd
Enter the county of the subject property:  Frederick
Enter the report date:  2lk3j423
Enter the PRG reference number:  234342
Enter the tax account:  l2kj34


Please choose a unit of measurement for lot size:
1. Acres
2. Square Feet


Enter the number of your choice (1 or 2):  1
Enter the number of acres:  54


What is the shape of the parcel?
1. Rectangular
2. Irregular


Please enter the number corresponding to your choice:  1


...

Enter the number corresponding to your choice:  1
Enter the number of stories:  three
Enter the number of bedrooms:  four
Enter the number of full bathrooms:  two
Enter the number of half bathrooms:  two
Enter the deed Liber:  54
Enter the deed Folio:  123
Enter the map number:  54
Enter the grid number:  234
Enter the parcel number:  32
Enter the election district:  11th
Enter the census tract:  432.234.6453
Enter the total road frontage of the subject property (feet):  56


Please choose type of valuation:
1. Current Market Value 'As-Is'
2. Retrospective Market Value


Enter the number of your choice (1 or 2):  1
Enter the date of valuation:  43j2lk43


Please the subject's zoning value from the following list:
1. Agricultural
2. Village Center



Enter the number corresponding to your choice:  1
Prepared by:  John S. Molesworth
